# Required vs. Optional Fields in Pydantic
* Notebook completed by Adam Lang
* Date: 4/10/2024
* This notebook goes over Required vs. Optional Fields in Pydantic and the implementation of it.
* This is from the udemy course Pydantic V2 Essentials.


So far all of the data fields we had defined in our pydantic models were required. Let's look at some ways to define required vs. optional field typing.

In [14]:
# create pydantic models
from pydantic import BaseModel, ValidationError

class Circle(BaseModel):
  center: tuple[int, int]
  radius: int

In [15]:
Circle.model_fields

{'center': FieldInfo(annotation=tuple[int, int], required=True),
 'radius': FieldInfo(annotation=int, required=True)}

Now, change the `required=True` field.

In [16]:
class Circle(BaseModel):
  center: tuple[int, int] = (0, 0)
  radius: int

In [17]:
# try again
Circle.model_fields

{'center': FieldInfo(annotation=tuple[int, int], required=False, default=(0, 0)),
 'radius': FieldInfo(annotation=int, required=True)}

Changed `required=` to False and added default values

In [18]:
# create instance and define
Circle(radius=1)

Circle(center=(0, 0), radius=1)

## Works with dictionary and JSON deserialization

In [19]:
#create dict
data = {"radius": 1}
data_json = '{"radius": 1}'

In [20]:
# validate dict
Circle.model_validate(data)

Circle(center=(0, 0), radius=1)

In [21]:
# validate json
Circle.model_validate_json(data_json)

Circle(center=(0, 0), radius=1)

#### Provide a value for Circle

In [22]:
Circle(center=(1,1), radius=2)

Circle(center=(1, 1), radius=2)

## Reminder about Deserialization
* Pyndantic validates a model during deserialization process BEFORE putting into pydantic model instance.

In [23]:
try:
  Circle(center="100", radius=2)
except ValidationError as ex:
  print(ex)

1 validation error for Circle
center
  Input should be a valid tuple [type=tuple_type, input_value='100', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/tuple_type


In [24]:
# create model with correct center
c = Circle(radius=2)
c

Circle(center=(0, 0), radius=2)

#### By default Pydantic will validate what you give it....
* So below we can see we gave it the optional field string of "junk" and it validated it during deserialization. Obviously this is not best practices but demonstrates how pydantic works.
* As a developer it is your responsibility to assign the correct default data types.

In [27]:
class Circle(BaseModel):
  center: tuple[int, int] = "junk"
  radius: int

In [28]:
# instantiate model
Circle(radius=2)

Circle(center='junk', radius=2)

# Pydantic Data Pearls
* You can use an empty list in a class as below, Pydantic creates a new list everytime this data type is encountered

In [29]:
class Model(BaseModel):
  my_list: list[int] = []

## However, below is an example of WHAT NOT TO DO
* The problem we see below:
1. We defined a default function parameter as an empty list.
2. We are then appending to that list everytime the function is called.
3. We will see why this is bad in the cell below this, but we will see that this does not prevent you from continuing to modify the list globally and locally.

In [35]:
from time import time

def extend_list(user_list: list=[]):
  user_list.append(int(time()))
  return user_list

In [38]:
## create a global list
my_times = []
extend_list(my_times) # pass global empty list to function
#print it
my_times

[1712783492]

In [39]:
## assign empty function to new variable
my_times = extend_list()
my_times

[1712783493]

In [40]:
my_new_times = extend_list()
my_new_times

[1712783493, 1712783494]

We can see that the list continued to carry forward the data points from the function and from each new variable instatiation.

# Summary
* Pydantic allows you to set required vs. optional fields.
* Pydantic will validate data types everytime it deserializes data into a pydantic model.
* Pydantic allows you to assign any default data type you like so choose wisely.
* We will go over in another notebook more advanced ways to handle default data structures like lists and dicts and how pydantic elegantly handles this.